In [1]:
import requests
import pandas as pd

# Your API Key
API_KEY = '04220a7214974d03aea163f7dccd1535'

# List of cities to check for weather alerts
cities = ['Atlanta', 'New York', 'Los Angeles', 'Chicago', 'Miami', 'Dallas', 'Seattle']

# Store the results in a list
alerts_data = []

# Iterate through each city and fetch weather alerts
for city in cities:
    url = f"https://api.weatherbit.io/v2.0/alerts?city={city}&key={API_KEY}"
    response = requests.get(url)
    
    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()
        
        # Extract alerts if present
        if "alerts" in data and data["alerts"]:
            for alert in data["alerts"]:
                alert["city"] = city  # Add city name to each alert
                alerts_data.append(alert)
    else:
        print(f"Failed to fetch data for {city}. Status Code: {response.status_code}")

# Convert the alerts data into a DataFrame for better visualization
df = pd.DataFrame(alerts_data)

# Display the DataFrame
if not df.empty:
    print(df)
else:
    print("No alerts found for the specified cities.")


                                         description      effective_local  \
0  * WHAT...West winds 20 to 25 mph with gusts up...  2024-12-04T04:18:00   
1  * WHAT...The South Coast AQMD has issued a man...  2024-12-03T16:09:00   
2  * WHAT...West winds 20 to 25 mph with gusts up...  2024-12-04T09:04:00   
3  * WHAT...Dangerous rip currents.\n\n* WHERE......  2024-12-04T00:57:00   
4  Fog will continue to slowly lift and scatter o...  2024-12-04T09:51:00   

         effective_utc           ends_local             ends_utc  \
0  2024-12-04T09:18:00  2024-12-05T22:00:00  2024-12-06T03:00:00   
1  2024-12-04T00:09:00  2024-12-05T00:00:00  2024-12-05T08:00:00   
2  2024-12-04T15:04:00  2024-12-05T00:00:00  2024-12-05T06:00:00   
3  2024-12-04T05:57:00  2024-12-04T19:00:00  2024-12-05T00:00:00   
4  2024-12-04T17:51:00  2024-12-04T10:00:00  2024-12-04T18:00:00   

         expires_local          expires_utc          onset_local  \
0  2024-12-04T17:00:00  2024-12-04T22:00:00  2024-12-05T06:0

In [2]:
#pip install pandas google-cloud-bigquery requests

In [9]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\priya\Downloads\datamanagement-443620-c2447c7bc4d3.json"

In [11]:
import requests
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

# Your API Key
API_KEY = '04220a7214974d03aea163f7dccd1535'

# List of cities to check for weather alerts
cities = ['Atlanta', 'New York', 'Los Angeles', 'Chicago', 'Miami', 'Dallas', 'Seattle']

# Store the results in a list
alerts_data = []

# Iterate through each city and fetch weather alerts
for city in cities:
    url = f"https://api.weatherbit.io/v2.0/alerts?city={city}&key={API_KEY}"
    response = requests.get(url)
    
    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()
        
        # Extract alerts if present
        if "alerts" in data and data["alerts"]:
            for alert in data["alerts"]:
                alert["city"] = city  # Add city name to each alert
                alerts_data.append(alert)
    else:
        print(f"Failed to fetch data for {city}. Status Code: {response.status_code}")

# Convert the alerts data into a DataFrame
df = pd.DataFrame(alerts_data)

# Print the DataFrame to verify the data
if not df.empty:
    print(df)
else:
    print("No alerts found for the specified cities.")

# Load data into BigQuery
project_id = "datamanagement-443620"
dataset_id = "WeatherData"
table_id = "WeatherTable"

# Initialize BigQuery client
client = bigquery.Client()

# Define table schema
schema = [
    SchemaField("title", "STRING", mode="NULLABLE"),
    SchemaField("description", "STRING", mode="NULLABLE"),
    SchemaField("severity", "STRING", mode="NULLABLE"),
    SchemaField("effective", "TIMESTAMP", mode="NULLABLE"),
    SchemaField("expires", "TIMESTAMP", mode="NULLABLE"),
    SchemaField("city", "STRING", mode="NULLABLE"),  # Add city field
]

# Create the dataset if it doesn't exist
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
try:
    client.create_dataset(dataset_ref, exists_ok=True)
    print(f"Dataset {dataset_id} created or already exists.")
except Exception as e:
    print(f"Error creating dataset: {e}")

# Create the table if it doesn't exist
table_ref = bigquery.Table(f"{project_id}.{dataset_id}.{table_id}", schema=schema)
try:
    client.create_table(table_ref, exists_ok=True)
    print(f"Table {table_id} created or already exists.")
except Exception as e:
    print(f"Error creating table: {e}")

# Load data into BigQuery
if not df.empty:
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()  # Wait for the job to complete
    print("Data loaded into BigQuery successfully!")
else:
    print("No data to load into BigQuery.")


                                         description      effective_local  \
0  * WHAT...West winds 20 to 25 mph with gusts up...  2024-12-04T04:18:00   
1  * WHAT...The South Coast AQMD has issued a man...  2024-12-03T16:09:00   
2  * WHAT...West winds 20 to 25 mph with gusts up...  2024-12-04T09:04:00   
3  * WHAT...Dangerous rip currents.\n\n* WHERE......  2024-12-04T00:57:00   
4  Fog will continue to slowly lift and scatter o...  2024-12-04T09:51:00   

         effective_utc           ends_local             ends_utc  \
0  2024-12-04T09:18:00  2024-12-05T22:00:00  2024-12-06T03:00:00   
1  2024-12-04T00:09:00  2024-12-05T00:00:00  2024-12-05T08:00:00   
2  2024-12-04T15:04:00  2024-12-05T00:00:00  2024-12-05T06:00:00   
3  2024-12-04T05:57:00  2024-12-04T19:00:00  2024-12-05T00:00:00   
4  2024-12-04T17:51:00  2024-12-04T10:00:00  2024-12-04T18:00:00   

         expires_local          expires_utc          onset_local  \
0  2024-12-04T17:00:00  2024-12-04T22:00:00  2024-12-05T06:0

In [23]:
#pip install db-dtypes
!pip install google-cloud-bigquery-storage

   ---------------------------------------- 0.0/240.0 kB ? eta -:--:--
   --- ----------------------------------- 20.5/240.0 kB 682.7 kB/s eta 0:00:01
   ---------------------------------- ----- 204.8/240.0 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 240.0/240.0 kB 3.7 MB/s eta 0:00:00


In [27]:
from google.cloud import bigquery
client = bigquery.Client()
query = f"""
    SELECT * FROM datamanagement-443620.WeatherData.WeatherTable
"""
# # Run the query and load data into a DataFrame
df = client.query(query).to_dataframe()

# Transformation 1: Combine effective_local and effective_utc into one datetime column
df["effective_combined"] = df["effective_local"].fillna(df["effective_utc"])

# Transformation 2: Filter alerts with severity "Severe"
filtered_df = df[df["severity"] == "Severe"]

# Transformation 3: Group by region and count alerts
transformed_df = filtered_df.groupby("regions").size().reset_index(name="alert_count")

# Display the transformed DataFrame
print(transformed_df)


Empty DataFrame
Columns: [regions, alert_count]
Index: []


In [29]:
from google.cloud import bigquery

client = bigquery.Client()

# List datasets in the project
datasets = list(client.list_datasets())
if datasets:
    print("Datasets in project:")
    for dataset in datasets:
        print(f" - {dataset.WeatherData}")
else:
    print("No datasets found.")


Datasets in project:


AttributeError: 'DatasetListItem' object has no attribute 'WeatherData'